Topic Analysis

In [ ]:
from google.colab import drive
import os

# gdrive_path='/content/gdrive/MyDrive/Bertopic/shared_work/'

# # This will mount your google drive under 'MyDrive'
# drive.mount('/content/gdrive', force_remount=True)
# # In order to access the files in this notebook we have to navigate to the correct folder
# os.chdir(gdrive_path)
# dataset_path = ''
# # Check manually if all files are present
# print(sorted(os.listdir()))

# To run from the common drive:
# dataset_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Models/bertopic_concatenated_chunks_stratified'
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/gdrive
['BERTopic.ipynb', 'BERTopic_female.ipynb', 'Data-PreProcessing.ipynb', 'Dataset_Statistics.ipynb', 'Regression.ipynb', 'Regression_RapSubset.ipynb', 'Regression_RapSubset_stratified.ipynb', 'Regression_RapSubset_stratified_topic.ipynb', 'betti2023_weat_wordsets.csv', 'chunk_5.csv', 'chunk_6.csv', 'concatenated_chunks.csv', 'genius_with_artists_first_chunk.csv', 'glove.6B.100d.txt', 'glove.6B.100d.txt.word2vec', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.6B.50d.txt', 'glove.6B.zip', 'model', 'model_new', 'my_model', 'mymodel', 'scweat_analysis (1).ipynb', 'top_words_by_topic.json', 'topic_analysis.ipynb', 'topics.json', 'topics.pkl', 'topics_for_bias_analysis.csv', 'updated_artists_with_gender_lyrics.csv', 'wasabi_artist_new_dartists.csv', 'wasabi_english_songs.csv', 'wasabi_new_dartists_balanced.csv', 'wasabi_new_songs_english_balanced_artists.csv', 'weat_results.csv']


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# from bertopic import BERTopic
# from umap import UMAP

# set path of the file
# csv_file_path = 'topics_for_bias_analysis.csv'
# df = pd.read_csv(dataset_path + csv_file_path)

In [ ]:
df.head()
df.describe()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,year,views,topic,topic_prob
count,537553.000000,537553.000000,537553.000000,537553.000000,5.375530e+05,537553.000000,537553.000000
mean,268776.000000,77024.914433,164628.276174,2002.695682,8.694508e+03,165.225821,0.684348
std,155178.328964,59943.213952,140773.031628,23.488690,1.028563e+05,156.126712,0.093572
min,0.000000,0.000000,0.000000,1.000000,0.000000e+00,-1.000000,0.050249
25%,134388.000000,25944.000000,46919.000000,1996.000000,7.700000e+01,28.000000,0.630626
50%,268776.000000,58987.000000,112588.000000,2008.000000,2.670000e+02,110.000000,0.693565
75%,403164.000000,125035.000000,275085.000000,2015.000000,1.092000e+03,269.000000,0.746582
max,537552.000000,217860.000000,522084.000000,2023.000000,1.757563e+07,541.000000,0.995553


In [ ]:
dataset = df[['title', 'artist', 'cleaned_lyrics', 'views', 'genre', 'year', 'gender', 'topic', 'topic_prob', 'topic_label']]
len(dataset['topic'].unique())

543

In [ ]:
def get_decade(year):
  if year >= 2000 and year < 2010:
    decade = '2000s'
  else:
    decade = str(year//100) + str(year%100)[0] + '0s'
  return decade

dataset['decade'] = dataset['year'].apply(get_decade)

<ipython-input-8-feedfd935843>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['decade'] = dataset['year'].apply(get_decade)


In [ ]:
dataset = dataset[dataset['topic'] != -1]
dataset = dataset[dataset['genre'] != 'misc']
top10topics = dataset['topic'].value_counts().nlargest(10).to_dict()
dataset['decade'] = dataset['year'].apply(get_decade)
mask = dataset['topic'].isin(top10topics)
top10 = dataset[mask]
top10
top10["topic"].unique()

array([  0,  98, 261,   2,  33,   3,  11,  85,   4,   1])

In [ ]:
top10

,title,artist,cleaned_lyrics,views,genre,year,gender,topic,topic_prob,topic_label,decade
1,Down and Out,Cam'ron,"\n\n\nUgh, Killa!\nBaby!\nKanye, this that 197...",144404,rap,2004,Male,0,0.675691,nigga_niggas_bitch,2000s
5,Losing Weight Pt. 2,Cam'ron,"\nAyo, fuck losing weight\nI'm back on these h...",32712,rap,2002,Male,0,0.776709,nigga_niggas_bitch,2000s
6,More Gangsta Music,Cam'ron,"\nGangsta Music, part two\nDipset\nKilla, Heat...",20419,rap,2004,Male,0,0.778837,nigga_niggas_bitch,2000s
7,Harlem Streets,Cam'ron,"\n\n\nKilla! (Killa!), Dipset, man\nAyo you kn...",22107,rap,2004,Male,0,0.740868,nigga_niggas_bitch,2000s
11,Get Em Daddy,Cam'ron,\nThey say beauty's in the eye of the beholder...,32413,rap,2006,Male,0,0.706905,nigga_niggas_bitch,2000s
...,...,...,...,...,...,...,...,...,...,...,...
537546,Broken Pt. 2,Michael W. Smith,"\n\nLord, bend down to listen to my prayer\nI'...",2,pop,2020,Male,1,0.795815,jesus_praise_lord,2020s
537547,Hope Pt. 3,Michael W. Smith,"\n\n""For I know the plans I have for you,"" dec...",2,pop,2020,Male,1,0.770242,jesus_praise_lord,2020s
537548,Blessing Pt. 5,Michael W. Smith,"\n\nYou, Lord, are forgiving and good\nAboundi...",2,pop,2020,Male,1,0.745915,jesus_praise_lord,2020s
537549,For The Girls,Hayley Kiyoko,"\n\n\nI'm done\nTake it deep, join the fun\nJu...",2,pop,2022,Female,3,0.539879,dance_funky_dance dance,2020s


Here's a simplified overview of how BERTopic works with a collection of documents to generate topic labels:

Document Embeddings: BERTopic converts each document into a high-dimensional vector using language models from the transformers library, such as BERT or Sentence-BERT. These embeddings capture the semantic meaning of the text.

Dimensionality Reduction: To manage the complexity of high-dimensional data and enhance clustering, BERTopic applies dimensionality reduction techniques, typically UMAP (Uniform Manifold Approximation and Projection), on these embeddings. This step helps to preserve the essential structure of the data while making it more tractable for clustering.

Clustering: Next, the reduced embeddings are clustered using a clustering algorithm, often HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise). Each cluster of documents represents a potential topic. The idea is that documents that are closer in the embedding space share thematic similarities and thus belong to the same topic.

Topic Creation and Refinement: BERTopic then reviews the clusters to identify and refine topics. This may involve merging similar clusters or removing outliers to ensure that each cluster represents a distinct topic accurately.

Label Generation: For each identified topic, BERTopic generates labels using a class-based TF-IDF (Term Frequency-Inverse Document Frequency) approach. This method identifies words that are uniquely representative of each topic, based on their frequency within the topic's documents compared to their frequency across all documents. The generated labels consist of words that capture the essence of each topic.

Topic Labels for Documents: Finally, each document in the dataset is associated with the topic of its closest cluster, and by extension, the topic label that summarizes that cluster. This way, BERTopic provides a comprehensive overview of the themes present in the dataset by assigning topic labels that reflect the content of the documents.

In [ ]:
dataset.describe()# Function to sample rows from each group
def sample_rows(group):
    return group.sample(n=50, replace=False) if len(group) >= 50 else group

# Filter the dataset for the top 10 topics
top10_topics = dataset['topic'].value_counts().nlargest(10).index
filtered_dataset = dataset[dataset['topic'].isin(top10_topics)]

# Randomly select 20 rows from each group with the same topic
sampled_dataset = filtered_dataset.groupby('topic').apply(sample_rows).reset_index(drop=True)

# Display the sampled dataset
print(sampled_dataset)


                                                 title        artist  \
0                                         Gunz-N-Rosez       Papoose   
1                                               DadBod         Logic   
2                                      What Ya Gone Do        Twista   
3                                       Infinity F.U.s       Matlock   
4                                 Anything Is Possible     Too $hort   
..                                                 ...           ...   
495                                 Hit That Jive Jack   Diana Krall   
496  Revolver Madonna vs. David Guetta One Love Rem...       Madonna   
497                        S-Deezy n Friends Phone Rap  Shane Dawson   
498                    Hello Good Morning Eminem Remix         Diddy   
499                                    Whats In My Pot  Dizzy Wright   

                                        cleaned_lyrics   views genre  year  \
0    Lyricism and life guns and roses\nRob execs ta...   

In BERTopic, the topic labels are not necessarily the most frequent words in the documents (or lyrics, in your case) of that topic. Instead, they are words that are most representative and distinguishing for the topic, determined through a class-based TF-IDF approach. This method prioritizes words that are uniquely significant to a topic relative to other topics, rather than just frequent.

Here’s a bit more detail on how this works:

Term Frequency (TF) measures how often a word appears within documents of the same topic. A word that appears more frequently in documents of a particular topic would have a higher term frequency.

Inverse Document Frequency (IDF) diminishes the weight of terms that appear very frequently across all topics and increases the weight of terms that occur rarely. The rationale is that frequent terms across all documents (or topics) are less informative and should be given lower importance.

Class-based TF-IDF combines these concepts but applies them in the context of each topic as a class. It looks for words that are frequent in a particular topic but not as common across other topics. This way, it identifies words that are uniquely significant to each topic.

This approach can highlight words that, while perhaps not the absolute most frequent within the documents of a topic, are more characteristic of the topic when considering the dataset as a whole. As a result, the labels generated for each topic are designed to be more meaningful and descriptive, capturing the essence of the topic rather than just listing common words.

For example, in a dataset of song lyrics, the word "love" might be very frequent across many topics. However, if a particular topic focuses on "heartbreak," the class-based TF-IDF might prioritize words like "goodbye," "tears," or "broken" for the topic label, assuming these words are relatively more unique to the "heartbreak" topic and less frequent in others. This method helps in making the topic labels more informative and useful for understanding the thematic focus of each topic.



for example niggas niggas bitch, those label words are not the top 1 2 frequent works there are some works like justm ya say, which are not stop words but it is also not uniquely representative for the docuemtns and lyrics. like the song "real OG"

for jesus praise lord, the lyrics documents also contains a lot of words like love, but it can be love for anyone, not only jesus, there are only words like god, but the inverse docuemtn frequency is low, indicating this word is also that unique and distinguishable (a lot of song use phrase oh my god that has nothing to do with jesus and lord).


the same goes for all other topics. but the label words always is present in the topic's document cluster.



body girl baby: body is not even in the topic 10 most frequent words.

"military time_military_time military": ilitary not in top 50 words

problematic labels: ayy ayy_change_long sentiment , heartache_bah_bah bah, maybe since the words like ayy and bah are really unque , so the bertopic model wrong thought that it is also representitive cuz of the inverse doc frequency\
em youre_worth em_ahahah also problematic.

the words for each topic's label are selected based on their class-based TF-IDF scores. However, BERTopic incorporates additional logic to ensure the selected words are representative and informative. It's not solely about picking the words with the absolute highest scores but also about choosing words that, together, best characterize the topic.

if I can have the tf idf score of words would be helpful



# Top Word Analysis

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import json
import nltk

# Function to find top words in lyrics
def find_top_words(lyrics, n=50):
    vectorizer = CountVectorizer(stop_words='english')
    word_matrix = vectorizer.fit_transform([lyrics])
    word_counts = word_matrix.toarray().flatten()
    word_features = vectorizer.get_feature_names_out()
    count_pairs = list(zip(word_features, word_counts))
    top_words = sorted(count_pairs, key=lambda x: x[1], reverse=True)[:n]
    return top_words

# Prepare data for JSON
data_for_json = {}

for topic in sampled_dataset['topic_label'].unique():
    topic_lyrics = " ".join(sampled_dataset[sampled_dataset['topic_label'] == topic]['cleaned_lyrics'])
    top_words_for_topic = find_top_words(topic_lyrics, n=50)
    data_for_json[str(topic)] = [{"word": word, "count": int(count)} for word, count in top_words_for_topic]

# Write to JSON file
with open(json_file_path, "w") as json_file:
    json.dump(data_for_json, json_file, indent=4)

json_file_path
# this is the top words frequency analysis

NameError: name 'json_file_path' is not defined

In [ ]:
!pip install pandas bertopic

TF-IDF (Term Frequency-Inverse Document Frequency) is a statistical measure used to evaluate the importance of a word to a document in a collection or corpus. It increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. TF-IDF is used in various applications like information retrieval and text mining.

c-TF-IDF (Class-based Term Frequency-Inverse Document Frequency), as used in BERTopic, adapts the traditional TF-IDF approach to work on a class level (in this case, topics). While TF-IDF typically operates at the document level across the entire corpus, c-TF-IDF calculates scores based on the term's frequency within a specific class (topic) compared to its distribution across all classes (topics). This method is particularly suited for topic modeling because it helps identify words that are not only frequent within a topic but also distinctively associated with that topic compared to others.

When you use model.get_topic(topic_number), the scores associated with each word reflect how important those words are for the specified topic, based on their c-TF-IDF calculation. These scores are useful for understanding the significance and uniqueness of words within topics identified by BERTopic. They are not traditional TF-IDF scores, which would be calculated without regard to topic structure and would be based on the word's presence across the entire dataset.

The c-TF-IDF scores help in highlighting terms that are uniquely significant to a topic, making them highly relevant for interpreting the topics discovered by BERTopic.

In [ ]:
from bertopic import BERTopic
import pandas as pd


model = BERTopic.load("model/MyModelNew/bertopic_concatenated_chunks_stratified")

# To get general information about all topics
info = model.get_topic_info()
print(info)

# To get the most representative words and their c-TF-IDF scores for a specific topic
topic_number = 0  # Example topic number
topic_words = model.get_topic(topic_number)
print(f"Most representative words and their scores for topic {topic_number}:")
print(topic_words)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

     Topic  Count                                               Name  \
0       -1  79304                               -1_love_im_dont_just   
1        0   7089                          0_nigga_niggas_bitch_shit   
2        1   1151                           1_jesus_praise_lord_holy   
3        2    974                               2_body_girl_baby_sex   
4        3    823                     3_dance_funky_dance dance_funk   
..     ...    ...                                                ...   
538    537     10              537_papa dont_papa_dont papa_toulouse   
539    538     10          538_wish_alright wish_wish tell_wish wish   
540    539     10  539_stand theyll_theyll stop_dont stand_gotta ...   
541    540     10        540_feel night_shadow_echo shadow_sit watch   
542    541     10                 541_noma_baba noma_baba_true heart   

                                        Representation  Representative_Docs  
0    [love, im, dont, just, know, oh, baby, youre, ...   

In [ ]:
top10_topics = dataset['topic'].value_counts().nlargest(10).index
topic_info = model.get_topic_info()

# Iterate over the top 10 topics
for topic_number in top10_topics:
    # Retrieve the topic label from the 'topic_info' DataFrame
    topic_label = topic_info[topic_info['Topic'] == topic_number]['Name'].values[0]

    # Retrieve the most representative words and their scores for the current topic
    topic_words_scores = model.get_topic(topic_number)

    # Print the topic label
    print(f"Topic label: {topic_label}")

    # Print each word and its corresponding c-TF-IDF score
    for word, score in topic_words_scores:
        print(f"{word}: {score}")

    # Add an empty line for better readability between topics
    print("\n")

Topic label: 0_nigga_niggas_bitch_shit
nigga: 0.005668254592840408
niggas: 0.005440166100871997
bitch: 0.004096094991031708
shit: 0.003805092244781979
fuck: 0.0037382588560453383
em: 0.0026590152866240242
money: 0.002634112321645001
ass: 0.002598596041345901
bitches: 0.0025326330634300336
aint: 0.0024592409943047506


Topic label: 2_body_girl_baby_sex
body: 0.00745922143452134
girl: 0.006751413063303796
baby: 0.005754223386395557
sex: 0.005125381128622596
yeah: 0.00497301113213543
wanna: 0.004502284218854553
yeah yeah: 0.004439478754062117
ooh: 0.0039741088594884575
ima: 0.003511711161320698
make: 0.0034132449588623035


Topic label: 11_tears_heart_wish_goodbye
tears: 0.006728059293429047
heart: 0.006027646650418587
wish: 0.005025114957016608
goodbye: 0.004985711368797596
dont want: 0.004962691086331912
dont: 0.004886653481264878
love: 0.0047991154148906735
walk away: 0.0045611152228384145
away: 0.00436424340208919
hurt: 0.004333572434081116


Topic label: 1_jesus_praise_lord_holy
jesu

The score you see printed alongside each word when using BERTopic's get_topic method is based on a metric known as class-based Term Frequency-Inverse Document Frequency (c-TF-IDF).
BERTopic employs c-TF-IDF to identify words that are not only frequent in a specific topic's documents but also provide a significant amount of information about that topic in relation to the entire corpus. The "class" in c-TF-IDF refers to the topic itself.